https://dadev.tistory.com/entry/Python-Pillow-%EC%9D%B4%EC%9A%A9%ED%95%9C-%EC%9D%B4%EB%AF%B8%EC%A7%80%EC%97%90-%ED%85%8D%EC%8A%A4%ED%8A%B8-%EC%B6%94%EA%B0%80

# 텍스트를 이미지에 렌더해보기

In [2]:
from PIL import Image, ImageFont, ImageDraw
from IPython.display import display

In [3]:
topPos = r'E:\GithubProjects\MukJumYee'
backgroundColor = (255, 255, 255)
fontColor = (0, 0, 0)

In [4]:
def loadFont(ttfPos : str, fontSize : int = 20):
	return ImageFont.truetype(font=ttfPos, size=fontSize)

fonts = {
	'바탕체' : loadFont(fr"{topPos}\fonts\batang.ttc"),
	'고딕체' : loadFont(fr"{topPos}\fonts\Hancom Gothic Regular.ttf"),
	'고딕체-굵음' : loadFont(fr"{topPos}\fonts\Hancom Gothic Bold.ttf"),
	'굴림체' : loadFont(fr"{topPos}\fonts\gulim.ttc"),
}

In [5]:
img = Image.new(mode="RGBA", size=(30,30), color=backgroundColor)
display(img)

In [6]:
def drawText(targetImage, text, font):
	out = ImageDraw.Draw(targetImage)
	out.text((5, 5), text, fontColor, font)
	return targetImage

In [7]:
display(drawText(img, "안", fonts['바탕체']))

In [8]:
def getImage(text, font):
    img = Image.new(mode="RGBA", size=(30,30), color=backgroundColor)
    drawText(img, text, font)
    return img

In [9]:
getImage('뷁', fonts['바탕체'])

In [10]:
getImage('뷁', fonts['고딕체'])

In [11]:
getImage('뷁', fonts['고딕체-굵음'])

In [12]:
getImage('뷁', fonts['굴림체'])

# 한글 목록 가져오기

In [13]:
korsList = []
with open(fr'{topPos}\kor.txt', 'r',encoding='utf8') as f:
    korsList = list(map(lambda x : x.strip(), f.readlines()))

In [14]:
len(korsList)

11172

In [15]:
fontNames = list(fonts.keys())
fontNames

['바탕체', '고딕체', '고딕체-굵음', '굴림체']

In [16]:
fontList = []

# 원 핫 인코딩 만들기  
- [1. 0. 0. 0. ... 0.] 같은 거  

In [17]:
import numpy as np

In [26]:
mapObj = { x:i for i, x in enumerate(korsList)}

def OneHotEncoding(data):
	ReturnObj = []

	ObjLen = len(mapObj)
	for label in data:
		temp = np.array([0. for _ in range(ObjLen)])
		temp[mapObj[label]] = 1.
		ReturnObj.append(temp)
	return np.array(ReturnObj)[0]

mapObjSwap = dict([(value, key) for key, value in mapObj.items()])

def OneHotDecoding(arr):
	maxPercent = 0
	result = "?"
	for i, percent in enumerate(arr):
		if percent > maxPercent:
			maxPercent = percent
			result = mapObjSwap[i]
	return (result, maxPercent)

In [33]:
temp = OneHotEncoding('가')
temp

array([1., 0., 0., ..., 0., 0., 0.])

In [34]:
OneHotDecoding(temp)

('가', 1.0)

# 데이터 셋 만들기

왜 이미지를 저장해서 안 쓰나요?  
직접 실험해본 결과 파일을 저장하고 불러오는 것보다 즉시 생성하여 램(변수)에 저장해두는 것이 훨씬 빠릅니다  

그리고 Pillow 의 이미지는 기본적으로 (r,g,b,a) a : alpha 으로 되어있기 때문에 이를 흑백(0~255) 로 변경해주어야 합니다


In [63]:
trainX = []
trainY = []

In [64]:
for i, txt in enumerate(korsList):
	tmp = OneHotEncoding(txt)
	for fontName in fontNames:
		trainX.append(np.asarray(getImage(txt, fonts[fontName]).convert('L')))
		trainY.append(tmp)
	print(f"\r{txt} | {int(((i+1)/len(korsList)) * 100)}%       ", end='')

힣 | 100%       

In [65]:
trainX = np.array(trainX)
trainY = np.array(trainY)

In [66]:
trainX.shape, trainY.shape

((44688, 30, 30), (44688, 11172))

# 모델

In [68]:
len(korsList)

11172

In [70]:
import tensorflow as tf

In [93]:
len(korsList)

11172

In [97]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(512, activation = 'tanh', input_shape = (30, 30,)),
	tf.keras.layers.Dense(256, activation = 'tanh',),
	tf.keras.layers.Dense(128, activation = 'tanh',),
	tf.keras.layers.Dense(64, activation = 'tanh',),
	tf.keras.layers.Dense(512, activation='tanh'),
	tf.keras.layers.Dense(11172, activation='sigmoid')
])
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_60 (Dense)            (None, 30, 512)           15872     
                                                                 
 dense_61 (Dense)            (None, 30, 256)           131328    
                                                                 
 dense_62 (Dense)            (None, 30, 128)           32896     
                                                                 
 dense_63 (Dense)            (None, 30, 64)            8256      
                                                                 
 dense_64 (Dense)            (None, 30, 512)           33280     
                                                                 
 dense_65 (Dense)            (None, 30, 11172)         5731236   
                                                                 
Total params: 5,952,868
Trainable params: 5,952,868
N

In [99]:
model.compile(
	optimizer = 'rmsprop',
	loss = 'categorical_crossentropy',
	metrics = ['accuracy']
)

In [102]:
trainX.shape, trainY.shape

((44688, 30, 30), (44688, 11172))

In [101]:
model.fit(trainX, trainY, epochs = 1)

ValueError: in user code:

    File "c:\Users\th070\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\th070\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\th070\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\th070\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\th070\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "c:\Users\th070\A